# 参数字段 JSON 转换
将 `demofixed.csv` 中 `参数(fixed)` 列内容解析为 JSON，写入新列 `参数(JSON)`，然后输出 `input.csv`。

In [1]:
!pip install pandas
print("✅ pandas 安装 / 可用")

✅ pandas 安装 / 可用


In [2]:
import pandas as pd
import re
import json

In [3]:
def parse_param_fixed(text):
    """把参数(fixed) 列里的文本（多行或 “键：值” 格式）解析成 JSON 对象：
    返回 list of {name, desc, valmap}"""
    if pd.isna(text) or not str(text).strip():
        return []
    lines = [l.strip() for l in str(text).splitlines() if l.strip()]
    params = []
    current = None
    for line in lines:
        # 如果是参数名行（形如 <xxx>）
        if re.match(r'^<.+?>$', line):
            if current:
                params.append(current)
            current = {"name": line, "desc": "", "valmap": {}}
        # 如果是 “key：value” 枚举行
        elif re.match(r'^[0-9]+[:：]', line):
            if current is None:
                # 没有参数名则忽略或创建一个默认名
                current = {"name": "", "desc": "", "valmap": {}}
            k, v = re.split(r'[:：]', line, 1)
            current["valmap"][k.strip()] = v.strip()
        # 其他作为描述行追加
        else:
            if current is None:
                # 若尚未有参数名行，用一个空名参数也可
                current = {"name": "", "desc": line, "valmap": {}}
            else:
                current["desc"] += line + " "
    if current:
        params.append(current)
    return params

In [4]:
# 主流程：读入 demofixed.csv，解析参数，写出 input.csv
df = pd.read_csv('demofixed.csv', encoding='utf-8')

def apply_parse(row):
    parsed = parse_param_fixed(row.get('参数(fixed)', ''))
    return json.dumps(parsed, ensure_ascii=False)

df['参数(JSON)'] = df.apply(apply_parse, axis=1)

output_path = 'input.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 已输出新 CSV：{output_path}")
df.head(5)

✅ 已输出新 CSV：input.csv


,章节,命令,命令标题,命令类型,命令格式,响应,示例命令,功能描述,备注,表数量,顺序,参数,参数（fixed）,参数(JSON)
0,1.0,ATI,获取模组厂商信息,执行,ATI,<CR><LF>OK<CR><LF>,ATI\n\nOK,获取模组厂商信息，包括厂家、型号和版本。\n命令格式,说明示例1,1.0,1.0,- `<manufacturer>`：模组厂商信息、产品名称、版本号\n\n- `<modu...,- `<manufacturer>`：模组厂商信息、产品名称、版本号\n\n- `<modu...,[]
1,2.0,AT+CSMS,选择短信服务,执行,AT+CSMS,<CR><LF>OK<CR><LF>,AT+CSMS\n\nOK,用于支持的短消息包括：发送（SMS-MO）、接收（SMS-MT）、小区广播（SMS-CB）。...,说明示例2,1.0,2.0,- `<service>`：0：GSM03.40 and GSM03.41；SMS相关AT指...,- `<service>`：\n - `0`: GSM03.40 and GSM03.41...,[]
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
